In [ ]:
import os
import logging
import traceback
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

load_dotenv(override=True)
API_KEYS = ["OPENAI_API_KEY", "GOOGLE_API_KEY", "ANTHROPIC_API_KEY"]
MODELS = {
    "ChatGPT": "gpt-4o-mini",
    "Gemini": "gemini-2.0-flash",
    "Claude": "claude-3-haiku-20240307"
}

api_keys = {key: os.getenv(key) for key in API_KEYS}
missing_keys = [key for key, value in api_keys.items() if not value]
if missing_keys:
    raise ValueError(f"Missing API keys: {', '.join(missing_keys)}")
    
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

openai = OpenAI(api_key=api_keys["OPENAI_API_KEY"])
google.generativeai.configure(api_key=api_keys["GOOGLE_API_KEY"])
claude = anthropic.Anthropic(api_key=api_keys["ANTHROPIC_API_KEY"])

SYSTEM_MESSAGE = (
    "You are a versatile, thoughtful, and precise assistant. "
    "Always prioritize fully understanding the user's request and ask clarifying questions when necessary. "
    "Adjust your tone and style based on the user's input."
)

force_dark_mode = """
function () {
    document.body.classList.add('dark');
}
"""

def error_handling(func, *args):
    try:
        yield from func(*args)
    except Exception as e:
        logging.error(f"Streaming error: {e}\n{traceback.format_exc()}")
        yield f"[Error: {str(e)}]"
        
def stream_chatgpt(messages: str):
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": messages}
    ]
    stream = openai.chat.completions.create(
        model=MODELS["ChatGPT"],
        messages=messages,
        stream=True
    )
    response = ""
    for chunk in stream:
        content = chunk.choices[0].delta.content or ""
        response += content
        yield response
        
def stream_gemini(messages: str):
    gemini = google.generativeai.GenerativeModel(
        model_name=MODELS["Gemini"],
        system_instruction=SYSTEM_MESSAGE
    )
    stream = gemini.generate_content(messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.text or ""
        yield response
        
def stream_claude(messages: str):
    result = claude.messages.stream(
        model=MODELS["Claude"],
        max_tokens=1000,
        temperature=0.7,
        system=SYSTEM_MESSAGE,
        messages=[{"role": "user", "content": messages}]
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response
            
def stream_model(messages: str, model: str):
    logging.info(f"Request received for model: {model}")
    if model == "ChatGPT":
        yield from error_handling(stream_chatgpt, messages)
    elif model == "Gemini":
        yield from error_handling(stream_gemini, messages)
    elif model == "Claude":
        yield from error_handling(stream_claude, messages)
    else:
        error_msg = f"Unknown model '{model}'. Available models: {', '.join(MODELS.keys())}"
        logging.error(error_msg)
        yield error_msg

def main():
    interface = gr.Interface(
        fn=stream_model,
        inputs=[
            gr.Textbox(label="Your message:", lines=6),
            gr.Dropdown(list(MODELS.keys()), label="Select model")
        ],
        outputs=gr.Textbox(label="Response:", lines=8),
        flagging_mode="never",
        js=force_dark_mode,
        title="My first Gradio"
    )
    interface.launch(share=True, inbrowser=True)
    
if __name__ == "__main__":
    main()